In [43]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import datetime
import psycopg2
from sqlalchemy import create_engine
import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
#macbook = os.path.expanduser('~')
dropbox_dir = r'/Users/kendraziegler/Dropbox (Palmer Lab)/Palmer Lab/Shae Ziegler/Suzanne_Mitchell_U01'
df_fits = pd.read_csv(dropbox_dir + '/fits.csv')
df_locomotor = pd.read_csv(dropbox_dir + '/locomotor.csv')
df_wfu = pd.read_csv(dropbox_dir + '/wfu_master.csv')
df_fecal_boli = pd.read_csv(dropbox_dir + '/fecal_boli.csv', dtype = str)

In [45]:
df_fits

,sex,cohort,rfid,coat_color,shipping_box,housing_box,order_in_box,squad_num,box_color,opertant_box,...,expon_adj_R2,expon_AIC,quad_k,quad_R2,quad_adj_R2,quad_AIC,linear_k,linear_R2,linear_adj_R2,linear_AIC
0,m,C02,933000320045751,brown_hood,5,40,2,4,blue,8,...,0.725429,36.342187,0.272698,-1.349954,0.000000,50.562703,6.678471,-0.035885,0.000000,45.647864
1,f,C02,933000320045756,black,11,54,2,5,blue,12,...,0.000000,42.756235,0.259993,-5.508170,0.000000,50.962255,6.338652,-2.516639,0.000000,47.268939
2,f,C02,933000320045757,brown_hood,11,18,2,2,blue,12,...,0.000000,42.401441,0.262557,-4.489528,0.000000,52.609082,6.564409,-2.148642,0.000000,49.273856
3,f,C02,933000320045759,brown_hood,10,52,2,5,blue,8,...,0.920194,32.324584,0.265759,0.378190,0.222737,45.981682,6.163581,0.814850,0.768562,38.712880
4,m,C02,933000320045760,brown,5,39,2,4,blue,6,...,0.879992,35.471797,0.247963,0.533578,0.416973,44.955831,5.690875,0.833890,0.792363,38.761188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,m,C04,933000320047963,black_hood,1,12,2,1,red,8,...,0.378722,39.455651,0.438252,0.506212,0.341616,39.745699,6.498826,0.554768,0.406358,39.228146
391,f,C04,933000320047964,black_hood,6,34,1,3,red,3,...,0.292852,44.912651,0.225828,0.333280,0.166600,45.898297,4.924056,0.460635,0.325794,44.626432
392,m,C04,933000320047968,brown_hood,1,27,1,3,blue,5,...,0.718459,40.902493,0.213945,0.611274,0.514093,44.176931,4.788950,0.761658,0.702073,41.241926
393,m,C04,933000320047973,black,1,12,1,1,red,7,...,0.897968,32.911002,0.222286,0.833993,0.792491,37.170351,4.822055,0.948307,0.935384,30.170114


In [46]:
def dd_fits():
    df_fits.rename(columns={'coat_color':'dd_coat_color','shipping_box':'dd_shipping_box','housing_box':'dd_housing_box',
                            'order_in_box':'dd_order_in_box','squad_num':'dd_squad_num','box_color':'dd_box_color',
                            'opertant_box':'dd_opertant_box','id_check':'dd_id_check','subject_num':'dd_subject_num',
                            's_or_ns':'dd_s_or_ns','missing_indiff_point':'dd_missing_indiff_point',
                            'missing_indiff_count':'dd_missing_indiff_count','indiff_0':'dd_indiff_0',
                            'indiff_2':'dd_indiff_2','indiff_4':'dd_indiff_4','indiff_8':'dd_indiff_8',
                            'indiff_16':'dd_indiff_16','indiff_24':'dd_indiff_24','hyperbolic_k':'dd_hyperbolic_k',
                            'hyperbolic_lnk':'dd_hyperbolic_lnk','hyperbolic_b':'dd_hyperbolic_b',
                            'hyperbolic_R2':'dd_hyperbolic_R2','hyperbolic_adj_R2':'dd_hyperbolic_adj_R2',
                            'hyperbolic_AIC':'dd_hyperbolic_AIC','AUC_traditional':'dd_AUC_traditional',
                            'AUC_normal':'dd_AUC_normal','hyperboloid_k':'dd_hyperboloid_k',
                            'hyperboloid_s':'dd_hyperboloid_s','hyperboloid_R2':'dd_hyperboloid_R2',
                            'hyperboloid_adj_R2':'dd_hyperboloid_adj_R2','hyperboloid_AIC':'dd_hyperboloid_AIC',
                            'quasi_H_k_beta':'dd_quasi_H_k_beta','quasi_H_s_delta':'dd_quasi_H_s_delta',
                            'quasi_H_R2':'dd_quasi_H_R2','quasi_H_adj_R2':'dd_quasi_H_adj_R2',
                            'quasi_H_AIC':'dd_quasi_H_AIC','expon_k':'dd_expon_k','expon_R2':'dd_expon_R2',
                            'expon_adj_R2':'dd_expon_adj_R2','expon_AIC':'dd_expon_AIC','quad_k':'dd_quad_k',
                            'quad_R2':'dd_quad_R2','quad_adj_R2':'dd_quad_adj_R2','quad_AIC':'dd_quad_AIC',
                            'linear_k':'dd_linear_k','linear_R2':'dd_linear_R2','linear_adj_R2':'dd_linear_adj_R2',
                            'linear_AIC':'dd_linear_AIC'},inplace=True)
dd_fits()
list(df_fits.columns.values)
#df_fits['cohort'].value_counts()


['sex',
 'cohort',
 'rfid',
 'dd_coat_color',
 'dd_shipping_box',
 'dd_housing_box',
 'dd_order_in_box',
 'dd_squad_num',
 'dd_box_color',
 'dd_opertant_box',
 'dd_id_check',
 'dd_subject_num',
 'dd_s_or_ns',
 'dd_missing_indiff_point',
 'dd_missing_indiff_count',
 'dd_indiff_0',
 'dd_indiff_2',
 'dd_indiff_4',
 'dd_indiff_8',
 'dd_indiff_16',
 'dd_indiff_24',
 'dd_hyperbolic_k',
 'dd_hyperbolic_lnk',
 'dd_hyperbolic_b',
 'dd_hyperbolic_R2',
 'dd_hyperbolic_adj_R2',
 'dd_hyperbolic_AIC',
 'dd_AUC_traditional',
 'dd_AUC_normal',
 'dd_hyperboloid_k',
 'dd_hyperboloid_s',
 'dd_hyperboloid_R2',
 'dd_hyperboloid_adj_R2',
 'dd_hyperboloid_AIC',
 'dd_quasi_H_k_beta',
 'dd_quasi_H_s_delta',
 'dd_quasi_H_R2',
 'dd_quasi_H_adj_R2',
 'dd_quasi_H_AIC',
 'dd_expon_k',
 'dd_expon_R2',
 'dd_expon_adj_R2',
 'dd_expon_AIC',
 'dd_quad_k',
 'dd_quad_R2',
 'dd_quad_adj_R2',
 'dd_quad_AIC',
 'dd_linear_k',
 'dd_linear_R2',
 'dd_linear_adj_R2',
 'dd_linear_AIC']

In [5]:
df_locomotor

,experiment,rfid,cohort,time,file_num,cage,sample,total_distance_x,total_distance_y,horizontal_activity_count,...,ambulatory_episode_peak_average_velocity,ambulatory_episode_median_velocity,left-front_time_legacy,right_front_time_legacy,left_rear_time_legacy,right_rear_time_legacy,margin_time_legacy,center_time_legacy,margin_distance_legacy,center_distance_legacy
0,U01-t1-gp1,933000320045924,C01,t1,gp1,3,120,430.86,468.96,2428,...,295.09,116.97,0.81,68.33,1.22,4.52,673.51,226.50,495.31,648.91
1,U01-t1-gp1,933000320046045,C02,t1,gp1,2,120,1109.68,1270.98,5250,...,447.70,261.87,2.36,8.27,2.16,19.40,346.25,553.76,754.50,2285.09
2,U01-t1-gp1,933000320046058,C01,t1,gp1,1,120,569.60,393.08,4209,...,272.30,177.73,0.00,0.20,7.37,1.57,169.89,730.12,313.14,1100.37
3,U01-t1-gp1,933000320046067,C01,t1,gp1,2,120,618.80,887.75,3748,...,349.34,184.81,4.89,19.55,2.25,10.15,522.49,377.54,723.80,1120.13
4,U01-t1-gp1,933000320046282,C01,t1,gp1,4,120,695.67,538.81,2972,...,257.18,139.19,2.05,17.52,5.40,2.34,232.15,667.88,471.61,1035.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,U01-t2-gp9,933000320047696,C04,t2,gp9,4,120,1233.80,1009.36,5262,...,386.37,285.33,105.93,44.64,239.59,74.24,884.67,15.34,2372.94,363.06
992,U01-t2-gp9,933000320187097,C05,t2,gp9,1,120,1578.30,1335.11,5151,...,444.02,300.89,21.99,20.95,25.19,191.81,723.13,176.87,2113.08,1156.71
993,U01-t2-gp9,933000320187155,C05,t2,gp9,2,120,823.30,562.31,3566,...,379.72,229.02,75.86,14.08,99.88,21.91,814.79,85.23,1257.33,527.50
994,U01-t2-gp9,933000320187194,C05,t2,gp9,3,120,1402.12,1163.00,5598,...,524.11,312.37,54.44,132.28,19.72,37.59,748.97,151.04,2153.31,916.42


In [47]:
def locomotor():
    df_locomotor.rename(columns={'experiment':'loco_experiment','time':'loco_time','file_num':'loco_file_num',
                                 'cage':'loco_cage','sample':'loco_sample','total_distance_x':'loco_total_distance_x',
                                 'total_distance_y':'loco_total_distance_y',
                                 'horizontal_activity_count':'loco_horizontal_activity_count',
                                 'ambulatory_activity_count':'loco_ambulatory_activity_count',
                                 'rest_time':'loco_rest_time','rest_episode_count':'loco_rest_episode_count',
                                 'movement_time':'loco_movement_time',
                                 'movement_episode_count':'loco_movement_episode_count',
                                 'ambulatory_time':'loco_ambulatory_time',
                                 'ambulatory_episode_count':'loco_ambulatory_episode_count',
                                 'stereotypy_time':'loco_stereotypy_time',
                                 'stereotypic_episode_count':'loco_stereotypic_episode_count',
                                 'stereotypic_activity_count':'loco_stereotypic_activity_count',
                                 'stereotypic_episode_activity_count':'loco_stereotypic_episode_activity_count',
                                 'vertical_episode_count':'loco_vertical_episode_count',
                                 'vertical_activity_count':'loco_vertical_activity_count',
                                 'vertical_activity_time':'loco_vertical_activity_time','jump_count':'loco_jump_count',
                                 'clockwise_revolutions':'loco_clockwise_revolutions',
                                 'counter_clockwise_revolutions':'loco_counter_clockwise_revolutions',
                                 'ambulatory_episode_average_velocity':'loco_ambulatory_episode_average_velocity',
                                 'ambulatory_episode_peak_average_velocity':'loco_ambulatory_episode_peak_average_velocity',
                                 'ambulatory_episode_median_velocity':'loco_ambulatory_episode_median_velocity',
                                 'left-front_time_legacy':'loco_left-front_time_legacy',
                                 'right_front_time_legacy':'loco_right_front_time_legacy',
                                 'left_rear_time_legacy':'loco_left_rear_time_legacy',
                                 'right_rear_time_legacy':'loco_right_rear_time_legacy',
                                 'margin_time_legacy':'loco_margin_time_legacy','center_time_legacy':'loco_center_time_legacy',
                                 'margin_distance_legacy':'loco_margin_distance_legacy',
                                 'center_distance_legacy':'loco_center_distance_legacy'},inplace = True) 
        
        
locomotor()        
list(df_locomotor.columns.values)
#df_locomotor['cohort'].value_counts()


['loco_experiment',
 'rfid',
 'cohort',
 'loco_time',
 'loco_file_num',
 'loco_cage',
 'loco_sample',
 'loco_total_distance_x',
 'loco_total_distance_y',
 'loco_horizontal_activity_count',
 'loco_ambulatory_activity_count',
 'loco_rest_time',
 'loco_rest_episode_count',
 'loco_movement_time',
 'loco_movement_episode_count',
 'loco_ambulatory_time',
 'loco_ambulatory_episode_count',
 'loco_stereotypy_time',
 'loco_stereotypic_episode_count',
 'loco_stereotypic_activity_count',
 'loco_stereotypic_episode_activity_count',
 'loco_vertical_episode_count',
 'loco_vertical_activity_count',
 'loco_vertical_activity_time',
 'loco_jump_count',
 'loco_clockwise_revolutions',
 'loco_counter_clockwise_revolutions',
 'loco_ambulatory_episode_average_velocity',
 'loco_ambulatory_episode_peak_average_velocity',
 'loco_ambulatory_episode_median_velocity',
 'loco_left-front_time_legacy',
 'loco_right_front_time_legacy',
 'loco_left_rear_time_legacy',
 'loco_right_rear_time_legacy',
 'loco_margin_time_le

In [48]:
def fecal_boli():
    df_fecal_boli.rename(columns = {'box':'fecal_box','date_1':'fecal_date_1','time_in_1':'fecal_time_in_1'
                                    ,'date_2':'fecal_date_2','time_in_2':'fecal_time_in_2',
                                    'change':'fecal_change'},inplace=True)
    
fecal_boli()
list(df_fecal_boli.columns.values)
#df_locomotor['cohort'].value_counts()


['rfid',
 'last_5',
 'cohort',
 'fecal_box',
 'sex',
 'fecal_date_1',
 'fecal_time_in_1',
 'fecal_boli_1',
 'fecal_date_2',
 'fecal_time_in_2',
 'fecal_boli_2',
 'fecal_change']

In [50]:
def wfu_columns():
    df_wfu.rename(columns = {'cohort',
 'sires',
 'dames',
 'labanimalid',
 'accessid',
 'sex',
 'rfid',
 'dob',
 'dow',
 'shipmentdate',
 'litternumber',
 'littersize',
 'coatcolor',
 'earpunch',
 'rack',
 'shipmentbox',
 'housingbox',
 'shipmentage',
 'weanage',
 'comments',
 'resolution',
 'last_5'},inplace = True)

list(df_wfu.columns.values)
#df_wfu['cohort'].value_counts()


['cohort',
 'sires',
 'dames',
 'labanimalid',
 'accessid',
 'sex',
 'rfid',
 'dob',
 'dow',
 'shipmentdate',
 'litternumber',
 'littersize',
 'coatcolor',
 'earpunch',
 'rack',
 'shipmentbox',
 'housingbox',
 'shipmentage',
 'weanage',
 'comments',
 'resolution',
 'last_5']

In [49]:
#Changes the int rfid and last_5 columns into strings
def reformat(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)
        df[col] = [val.replace('.0','') for val in df[col]]
        
reformat(df_fecal_boli, ["rfid", "last_5"])
reformat(df_fits, ["rfid"])
reformat(df_locomotor, ["rfid"])
reformat(df_wfu, ["rfid","housingbox"])
df_fecal_boli


,rfid,last_5,cohort,fecal_box,sex,fecal_date_1,fecal_time_in_1,fecal_boli_1,fecal_date_2,fecal_time_in_2,fecal_boli_2,fecal_change
0,933000320046058,46058,C01,1,M,2018-12-20,12:26:00,5,2018-12-21,13:09:00,4.0,-1.0
1,933000320046067,46067,C01,1,M,2018-12-20,12:26:00,8,2018-12-21,13:09:00,6.0,-2.0
2,933000320046282,46282,C01,2,M,2018-12-20,12:28:00,11,2018-12-21,13:10:00,7.0,-4.0
3,933000320045924,45924,C01,2,M,2018-12-20,12:27:00,8,2018-12-21,13:10:00,9.0,1.0
4,933000320046276,46276,C01,3,M,2018-12-20,12:51:00,5,2018-12-21,13:33:00,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
631,933000320124750,24750,C07,21,M,2020-10-12,12:42:00,0,2020-10-13,12:43:00,0.0,0.0
632,933000320124693,24693,C07,23,M,2020-10-12,13:04:00,0,2020-10-13,13:02:00,2.0,2.0
633,933000320124682,24682,C07,23,M,2020-10-12,13:05:00,3,2020-10-13,13:03:00,2.0,-1.0
634,933000320124669,24669,C07,24,M,2020-10-12,13:05:00,4,2020-10-13,13:03:00,6.0,2.0


In [8]:
df_wfu

,cohort,sires,dames,labanimalid,accessid,sex,rfid,dob,dow,shipmentdate,...,coatcolor,earpunch,rack,shipmentbox,housingbox,shipmentage,weanage,comments,resolution,last_5
0,C01,72539_1,72595_5,TJ008,73472_2,F,933000320045906,2018-09-24,2018-10-15,2018-10-30,...,BROWNHOOD,RB,D-F4,2.0,9999,36.0,21.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE,45906
1,C01,73358_1,73360_1,TJ027,73477_2,F,933000320045890,2018-09-25,2018-10-16,2018-10-30,...,BROWNHOOD,LM,D-G8,4.0,32,35.0,21.0,NaN,NaN,45890
2,C01,72539_1,72595_5,TJ003,73472_8,M,933000320045902,2018-09-24,2018-10-15,2018-10-30,...,ALBINO,RT,D-F2,2.0,9999,36.0,21.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE,45902
3,C01,72794_1,72775_4,TJ020,73475_4,F,933000320045904,2018-09-25,2018-10-16,2018-10-30,...,BROWN,RB,D-G2,2.0,9999,35.0,21.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE,45904
4,C01,72624_1,72596_4,TJ002,73471_5,F,933000320045908,2018-09-23,2018-10-15,2018-10-30,...,BLACK,LT,D-F2,2.0,9999,37.0,22.0,Pregnant female/Impregnated,REMOVE_FROM_EXCLUSION_AND_REPLACE,45908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,C07,933000320048190,933000320187377,NaN,NaN,F,933000320125436,2020-08-09,2020-09-04,NaN,...,BLACK,NaN,NaN,NaN,nan,NaN,26.0,NaN,NaN,25436
897,C07,933000320187153,933000320187383,NaN,NaN,F,933000320125437,2020-08-09,2020-09-04,NaN,...,BROWNHOOD,NaN,NaN,NaN,nan,NaN,26.0,NaN,NaN,25437
898,C07,933000320187217,933000320048245,NaN,NaN,F,933000320125440,2020-08-09,2020-09-04,NaN,...,BROWN,NaN,NaN,NaN,nan,NaN,26.0,NaN,NaN,25440
899,C07,933000320187249,933000320048234,NaN,NaN,M,933000320125443,2020-08-08,2020-09-04,NaN,...,BLACKHOOD,NaN,NaN,NaN,nan,NaN,27.0,NaN,NaN,25443


C07    291
C02    110
C06    100
C03    100
C04    100
C01    100
C05    100
Name: cohort, dtype: int64

In [10]:
wfu_rfid = df_wfu.last_5.unique()
locomotor_rfid = df_locomotor.rfid.unique()
missmatch = [x for x in wfu_rfid if x not in locomotor_rfid]
print(len(wfu_rfid))
print(len(locomotor_rfid))
len(missmatch)

901
500


901

In [56]:
df_fits_locomotor = pd.merge(df_wfu, df_locomotor, on=['rfid','cohort'],how = 'outer')
df_wfu_fecal = pd.merge(df_fits_locomotor, df_fecal_boli, on=['rfid','cohort'],how = 'outer')
df_all = pd.merge(df_wfu_fecal, df_fits, on=['rfid','cohort'],how = 'outer')
df_all

,cohort,sires,dames,labanimalid,accessid,sex_x,rfid,dob,dow,shipmentdate,...,dd_expon_adj_R2,dd_expon_AIC,dd_quad_k,dd_quad_R2,dd_quad_adj_R2,dd_quad_AIC,dd_linear_k,dd_linear_R2,dd_linear_adj_R2,dd_linear_AIC
0,C01,72539_1,72595_5,TJ008,73472_2,F,933000320045906,2018-09-24,2018-10-15,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C01,73358_1,73360_1,TJ027,73477_2,F,933000320045890,2018-09-25,2018-10-16,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C01,73358_1,73360_1,TJ027,73477_2,F,933000320045890,2018-09-25,2018-10-16,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C01,72539_1,72595_5,TJ003,73472_8,M,933000320045902,2018-09-24,2018-10-15,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C01,72794_1,72775_4,TJ020,73475_4,F,933000320045904,2018-09-25,2018-10-16,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,C06,NaN,NaN,NaN,NaN,NaN,933000320148178,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1444,C06,NaN,NaN,NaN,NaN,NaN,933000320148244,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1445,C06,NaN,NaN,NaN,NaN,NaN,933000320148227,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1446,C06,NaN,NaN,NaN,NaN,NaN,933000320148074,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
list(df_all.columns.values)

['cohort',
 'sires',
 'dames',
 'labanimalid',
 'accessid',
 'sex_x',
 'rfid',
 'dob',
 'dow',
 'shipmentdate',
 'litternumber',
 'littersize',
 'coatcolor',
 'earpunch',
 'rack',
 'shipmentbox',
 'housingbox',
 'shipmentage',
 'weanage',
 'comments',
 'resolution',
 'last_5_x',
 'loco_experiment',
 'loco_time',
 'loco_file_num',
 'loco_cage',
 'loco_sample',
 'loco_total_distance_x',
 'loco_total_distance_y',
 'loco_horizontal_activity_count',
 'loco_ambulatory_activity_count',
 'loco_rest_time',
 'loco_rest_episode_count',
 'loco_movement_time',
 'loco_movement_episode_count',
 'loco_ambulatory_time',
 'loco_ambulatory_episode_count',
 'loco_stereotypy_time',
 'loco_stereotypic_episode_count',
 'loco_stereotypic_activity_count',
 'loco_stereotypic_episode_activity_count',
 'loco_vertical_episode_count',
 'loco_vertical_activity_count',
 'loco_vertical_activity_time',
 'loco_jump_count',
 'loco_clockwise_revolutions',
 'loco_counter_clockwise_revolutions',
 'loco_ambulatory_episode_av

In [66]:
def format_columns(df_all):
    df_all = df_all.drop(['sex_x', 'sex_y','dd_coat_color','last_5_x','last_5_y','shipmentbox','comments','resolution','dd_order_in_box'], axis=1)
    

    df_all["sex"] = df_all["sex"].replace("m", "M", regex=True)
    df_all["sex"] = df_all["sex"].replace("f", "F", regex=True)
    #df_all.sex_x.fillna(df_all.sex_y, inplace=True)
    return df_all

    
df_all = format_columns(df_all)


In [67]:
#df_all['time'].value_counts()
list(df_all.columns.values)

['cohort',
 'sires',
 'dames',
 'labanimalid',
 'accessid',
 'rfid',
 'dob',
 'dow',
 'shipmentdate',
 'litternumber',
 'littersize',
 'coatcolor',
 'earpunch',
 'rack',
 'housingbox',
 'shipmentage',
 'weanage',
 'loco_experiment',
 'loco_time',
 'loco_file_num',
 'loco_cage',
 'loco_sample',
 'loco_total_distance_x',
 'loco_total_distance_y',
 'loco_horizontal_activity_count',
 'loco_ambulatory_activity_count',
 'loco_rest_time',
 'loco_rest_episode_count',
 'loco_movement_time',
 'loco_movement_episode_count',
 'loco_ambulatory_time',
 'loco_ambulatory_episode_count',
 'loco_stereotypy_time',
 'loco_stereotypic_episode_count',
 'loco_stereotypic_activity_count',
 'loco_stereotypic_episode_activity_count',
 'loco_vertical_episode_count',
 'loco_vertical_activity_count',
 'loco_vertical_activity_time',
 'loco_jump_count',
 'loco_clockwise_revolutions',
 'loco_counter_clockwise_revolutions',
 'loco_ambulatory_episode_average_velocity',
 'loco_ambulatory_episode_peak_average_velocity',


In [ ]:
df_all['housing_box'].equals(df_all['housingbox'])
